In [1]:
test_index = 0

#### testing

In [2]:
from load_data import *

In [3]:
# load_data()

## Loading the data

In [4]:
from load_data import *

In [5]:
X_train,X_test,y_train,y_test = load_data()

4
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Shine/shine131.jpg
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Rain/rain141.jpg


/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [6]:
len(X_train),len(y_train)

(843, 843)

In [7]:
len(X_test),len(y_test)

(280, 280)

## Test Modelling

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
class Test_Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv2d(1,64,5)
        self.c2 = nn.Conv2d(64,128,5)
        self.c3 = nn.Conv2d(128,256,5)
        self.fc4 = nn.Linear(256*10*10,256)
        self.fc6 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,4)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.c1(X)),(2,2))
        preds = F.max_pool2d(F.relu(self.c2(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.c3(preds)),(2,2))
#         print(preds.shape)
        preds = preds.view(-1,256*10*10)
        preds = F.relu(self.fc4(preds))
        preds = F.relu(self.fc6(preds))
        preds = self.fc5(preds)
        return preds

In [10]:
device = torch.device('cuda')

In [11]:
BATCH_SIZE = 32

In [12]:
IMG_SIZE = 112

In [13]:
model = Test_Model().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [14]:
EPOCHS = 12

In [15]:
from tqdm import tqdm

In [16]:
PROJECT_NAME = 'Weather-Clf'

In [17]:
import wandb

In [18]:
test_index += 1
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|          | 0/12 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 12/12 [00:08<00:00,  1.38it/s]


loss,1.07195
_runtime,15
_timestamp,1621350020
_step,323


loss,▆▆▆▆▆▅▆▇▆▆▅▆▅▆█▆▅▅▅▄▄▄▅▁▅▄▃▄▃▃▃▄▁▂▄▁▃▄▂▂
_runtime,▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇█
_timestamp,▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [19]:
for index in range(10):
    print(torch.argmax(preds[index]))
    print(y_batch[index])
    print('\n')

tensor(3, device='cuda:0')
tensor(1, device='cuda:0')


tensor(2, device='cuda:0')
tensor(2, device='cuda:0')


tensor(3, device='cuda:0')
tensor(3, device='cuda:0')


tensor(3, device='cuda:0')
tensor(3, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(0, device='cuda:0')


tensor(0, device='cuda:0')
tensor(3, device='cuda:0')


tensor(1, device='cuda:0')
tensor(0, device='cuda:0')


tensor(2, device='cuda:0')
tensor(2, device='cuda:0')


tensor(1, device='cuda:0')
tensor(0, device='cuda:0')




In [23]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        self.conv4 = nn.Conv2d(128,256,3)
        self.conv5 = nn.Conv2d(256,512,3)
        self.fc1 = nn.Linear(512,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,64)
        self.fc5 = nn.Linear(64,6)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(F.relu(self.conv2(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.conv3(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.conv4(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,512)
        preds = F.relu(self.fc1(preds))
        preds = F.relu(self.fc2(preds))
        preds = F.relu(self.fc3(preds))
        preds = F.relu(self.fc4(preds))
        preds = F.relu(self.fc5(preds))
        return preds

In [24]:
model = Test_Model().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [25]:
test_index += 1
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()

  0%|          | 0/12 [00:00<?, ?it/s]

torch.Size([32, 512, 1, 1])


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
for index in range(10):
    print(torch.argmax(preds[index]))
    print(y_batch[index])
    print('\n')